In [44]:
import math
from typing import TypedDict

import pandas as pd

from helpers import upload_to_github

Rating = TypedDict("Rating", {"rating": float, "event": str})
Player = TypedDict(
    "Player",
    {
        "name": str,
        "played": int,
        "wins": int,
        "losses": int,
        "rating": float,
        "rating_history": list[Rating],
        'opponent_value_sum': float,
        'opponent_value': float,
    },
)
Match = TypedDict(
    "Match",
    {
        "team_a": list[Player],
        "team_b": list[Player],
        "score_a": int,
        "score_b": int,
        "winner": list[Player],
        "loser": list[Player],
    },
)
MatchResult = TypedDict(
    "MatchResult",
    {
        "player_1_played": int,
        "player_1_wins": int,
        "player_1_losses": int,
        "player_1_rating": float,
        "player_1_opponent_value": float,
        "player_2_played": int,
        "player_2_wins": int,
        "player_2_losses": int,
        "player_2_rating": float,
        "player_2_opponent_value": float,
        "player_3_played": int,
        "player_3_wins": int,
        "player_3_losses": int,
        "player_3_rating": float,
        "player_3_opponent_value": float,
        "player_4_played": int,
        "player_4_wins": int,
        "player_4_losses": int,
        "player_4_rating": float,
        "player_4_opponent_value": float
    },
)
MatchResultRating = TypedDict(
    "MatchResultRating",
    {
        "player_1_rating": float,
        "player_2_rating": float,
        "player_3_rating": float,
        "player_4_rating": float,
        "player_1_opponent_value": float,
        "player_2_opponent_value": float,
        "player_3_opponent_value": float,
        "player_4_opponent_value": float,
        
    },
)


def create_player(name: str) -> Player:
    """
    Creates a new player with the given name and initializes their statistics.

    Args:
        name (str): The name of the player.

    Returns:
        dict[str, Any]: A dictionary representing the player with initialized stats.
    """
    return {
        "name": name,
        "played": 0,
        "wins": 0,
        "losses": 0,
        "rating": 1500,  # Initial Elo rating
        "opponent_value": 0,
        "opponent_value_sum": 0,
        "partner_value": 0,
        "partner_value_sum": 0,
        "rating_history": [],
    }


def create_match(
    team_a: list[Player],
    team_b: list[Player],
    score_a: int,
    score_b: int,
) -> Match:
    """
    Creates a match between two teams and records the scores and results.

    Args:
        team_a (list[dict[str, Any]]): The first team consisting of players.
        team_b (list[dict[str, Any]]): The second team consisting of players.
        score_a (int): The score of team A.
        score_b (int): The score of team B.

    Returns:
        dict[str, Any]: A dictionary representing the match details, including teams and scores.
    """
    return {
        "team_a": team_a,
        "team_b": team_b,
        "score_a": score_a,
        "score_b": score_b,
        "winner": team_a if score_a > score_b else team_b,
        "loser": team_b if score_a > score_b else team_a,
    }


def play_match(match: Match) -> MatchResult:
    """
    Processes the match results, updating player records and ratings based on the scores.

    Args:
        match (Match): The match details containing teams and scores.
    """
    match_result: MatchResult = {
        "player_1_played": 1,
        "player_1_wins": 0,
        "player_1_losses": 0,
        "player_1_rating": 0,
        "player_2_played": 1,
        "player_2_wins": 0,
        "player_2_losses": 0,
        "player_2_rating": 0,
        "player_3_played": 1,
        "player_3_wins": 0,
        "player_3_losses": 0,
        "player_3_rating": 0,
        "player_4_played": 1,
        "player_4_wins": 0,
        "player_4_losses": 0,
        "player_4_rating": 0,
    }

    # If team A wins
    if match["score_a"] > match["score_b"]:
        # record a win for player 1 and 2
        match_result["player_1_wins"] = 1
        match_result["player_2_wins"] = 1

        # record a loss for player 3 and 4
        match_result["player_3_losses"] = 1
        match_result["player_4_losses"] = 1

    # If team B wins
    else:
        # record a win for player 3 and 4
        match_result["player_3_wins"] = 1
        match_result["player_4_wins"] = 1

        # record a loss for player 1 and 2
        match_result["player_1_losses"] = 1
        match_result["player_2_losses"] = 1

    # Calculate rating
    rating = calculate_ratings(
        team_a=match["team_a"],
        team_b=match["team_b"],
        score_a=match["score_a"],
        score_b=match["score_b"],
    )

    match_result["player_1_rating"] = rating["player_1_rating"]
    match_result["player_2_rating"] = rating["player_2_rating"]
    match_result["player_3_rating"] = rating["player_3_rating"]
    match_result["player_4_rating"] = rating["player_4_rating"]
    match_result["opponent_value_A"] = rating["opponent_value_A"]
    match_result["opponent_value_B"] = rating["opponent_value_B"]
    match_result["partner_value_A_1"] = rating["partner_value_A_1"]
    match_result["partner_value_A_2"] = rating["partner_value_A_2"]
    match_result["partner_value_B_1"] = rating["partner_value_B_1"]
    match_result["partner_value_B_2"] = rating["partner_value_B_2"]


    return match_result


def calculate_ratings(
    team_a: list[Player],
    team_b: list[Player],
    score_a: int,
    score_b: int,
    k_factor: int = 32,
) -> MatchResultRating:
    """
    Updates the ratings for each player in two teams competing in a doubles match.

    Args:
        team_a (list[Player]): The first team consisting of players.
        team_b (list[Player]): The second team consisting of players.
        score_a (int): The score of team A.
        score_b (int): The score of team B.
        k_factor (int, optional): The K-factor for rating adjustment. Default is 32.

    """
    match_result_rating: MatchResultRating = {
        "player_1_rating": 0,
        "player_2_rating": 0,
        "player_3_rating": 0,
        "player_4_rating": 0,
        "opponent_value_A": 0,
        "opponent_value_B": 0,
        "partner_value_A": 0,
        "partner_value_B": 0,
    }

    rating_team_a = (team_a[0]["rating"] + team_a[1]["rating"]) / 2
    rating_team_b = (team_b[0]["rating"] + team_b[1]["rating"]) / 2

    match_result_rating["opponent_value_A"] = rating_team_b
    match_result_rating["opponent_value_B"] = rating_team_a

    match_result_rating["partner_value_A_1"] = team_a[1]["rating"]
    match_result_rating["partner_value_A_2"] = team_a[0]['rating']
    match_result_rating["partner_value_B_1"] = team_b[1]["rating"]
    match_result_rating["partner_value_B_2"] = team_b[0]['rating']

    expected_score_a = 1 / (1 + math.pow(10, (rating_team_b - rating_team_a) / 400))
    expected_score_b = 1 / (1 + math.pow(10, (rating_team_a - rating_team_b) / 400))

    actual_score_a = 1 if score_a > score_b else 0
    actual_score_b = 1 if score_b > score_a else 0

    margin_factor = 1 + abs(score_a - score_b) / 21

    # Update team A rating
    rating_change_team_a = (
        k_factor * margin_factor * (actual_score_a - expected_score_a)
    )
    match_result_rating["player_1_rating"] = rating_change_team_a
    match_result_rating["player_2_rating"] = rating_change_team_a

    # Update team B rating
    rating_change_team_b = (
        k_factor * margin_factor * (actual_score_b - expected_score_b)
    )
    match_result_rating["player_3_rating"] = rating_change_team_b
    match_result_rating["player_4_rating"] = rating_change_team_b

    # for player in team_b:
    #     rating_change = k_factor * margin_factor * (actual_score_b - expected_score_b)
    #     player["rating"] += rating_change

    return match_result_rating


def scaled_rating(rating: float) -> float:
    """
    Scales the player's rating to a 10-point scale.

    Args:
        rating (float): The raw rating of the player.

    Returns:
        float: The scaled rating between 0 and 10, rounded to one decimal place.
    """
    min_rating = 1000
    max_rating = 2000
    scaled = (rating - min_rating) / (max_rating - min_rating) * 10
    return round(max(0, min(scaled, 10)), 1)  # Keep the result between 0 and 10


def get_last_week_rating_change(rating_history: list[Rating]) -> float:
    """
    Calculates the change in rating from last available session (typically last week) to the current rating.

    Args:
        rating_history (list[Rating]): The rating history of a player.

    Returns:
        float: The change in rating from the last available session.
    """
    if len(rating_history) < 2:
        return 0  # No previous events to compare
    current = rating_history[-1]["rating"]
    previous = rating_history[-2]["rating"]
    return current - previous


def get_last_event_rating_change(rating_history: list[Rating]) -> float:
    """
    Calculates the change in rating from the last recorded event to the current rating.

    Args:
        rating_history (list[Rating]): The rating history of a player.

    Returns:
        float: The change in rating from the last event.
    """
    if len(rating_history) == 0:
        return 0  # No previous events to compare

    elif len(rating_history) == 1:
        return rating_history[0]["rating"] - 1500

    current = rating_history[-1]["rating"]
    previous = rating_history[-2]["rating"]
    return current - previous


player_dict: dict[str, Player] = {}

for badminton_date in ["2024-10-10", "2024-10-17", "2024-10-31", "2024-11-07"]:
    df = pd.read_csv(
        f"https://raw.githubusercontent.com/BadgerMinton/badgerminton-data/refs/heads/main/matches/{badminton_date}_match_results.csv"
    )

    # Create a Player "database"
    player_names = pd.concat(
        [df[col] for col in ["player_a_1", "player_a_2", "player_b_1", "player_b_2"]]
    ).unique()
    for player_name in player_names:
        if player_name not in player_dict:
            player = create_player(name=player_name)
            player_dict[player_name] = player

    for player_name in player_names:
        if "opponent_value" not in player_dict[player_name]:
            player_dict[player_name]["opponent_value"] = 0  # or an appropriate default value
        
        elif "partner_value" not in player_dict[player_name]:
            player_dict[player_name]["partner_value"] = 0  # or an appropriate default value


    for index, row in df.iterrows():
        # Create players
        player1 = player_dict[row["player_a_1"]]
        player2 = player_dict[row["player_a_2"]]
        player3 = player_dict[row["player_b_1"]]
        player4 = player_dict[row["player_b_2"]]

        # Create teams
        team_a = [player1, player2]  # Team A: Alice and Bob
        team_b = [player3, player4]  # Team B: Charlie and Diana

        # Create a match
        match = create_match(
            team_a, team_b, score_a=row["score_a"], score_b=row["score_b"]
        )

        # Play the match
        match_result = play_match(match)

        # Save the values
        # Player 1
        player_dict[row["player_a_1"]]["wins"] += match_result["player_1_wins"]
        player_dict[row["player_a_1"]]["losses"] += match_result["player_1_losses"]
        player_dict[row["player_a_1"]]["played"] += match_result["player_1_played"]
        player_dict[row["player_a_1"]]["rating"] += match_result["player_1_rating"]
        player_dict[row["player_a_1"]]["opponent_value_sum"] += match_result["opponent_value_A"] 
        player_dict[row["player_a_1"]]["opponent_value"] = player_dict[row["player_a_1"]]["opponent_value_sum"] / player_dict[row["player_a_1"]]["played"]
        player_dict[row["player_a_1"]]["partner_value_sum"] += match_result["partner_value_A_1"] 
        player_dict[row["player_a_1"]]["partner_value"] = player_dict[row["player_a_1"]]["partner_value_sum"] / player_dict[row["player_a_1"]]["played"]


        # Player 2
        player_dict[row["player_a_2"]]["wins"] += match_result["player_2_wins"]
        player_dict[row["player_a_2"]]["losses"] += match_result["player_2_losses"]
        player_dict[row["player_a_2"]]["played"] += match_result["player_2_played"]
        player_dict[row["player_a_2"]]["rating"] += match_result["player_2_rating"]
        player_dict[row["player_a_2"]]["opponent_value_sum"] += match_result["opponent_value_A"] 
        player_dict[row["player_a_2"]]["opponent_value"] = player_dict[row["player_a_2"]]["opponent_value_sum"] / player_dict[row["player_a_2"]]["played"]
        player_dict[row["player_a_2"]]["partner_value_sum"] += match_result["partner_value_A_2"] 
        player_dict[row["player_a_2"]]["partner_value"] = player_dict[row["player_a_2"]]["partner_value_sum"] / player_dict[row["player_a_2"]]["played"]

        # Player 3
        player_dict[row["player_b_1"]]["wins"] += match_result["player_3_wins"]
        player_dict[row["player_b_1"]]["losses"] += match_result["player_3_losses"]
        player_dict[row["player_b_1"]]["played"] += match_result["player_3_played"]
        player_dict[row["player_b_1"]]["rating"] += match_result["player_3_rating"]
        player_dict[row["player_b_1"]]["opponent_value_sum"] += match_result["opponent_value_B"] 
        player_dict[row["player_b_1"]]["opponent_value"] = player_dict[row["player_b_1"]]["opponent_value_sum"] / player_dict[row["player_b_1"]]["played"]
        player_dict[row["player_b_1"]]["partner_value_sum"] += match_result["partner_value_B_1"] 
        player_dict[row["player_b_1"]]["partner_value"] = player_dict[row["player_b_1"]]["partner_value_sum"] / player_dict[row["player_b_1"]]["played"]

        # Player 4
        player_dict[row["player_b_2"]]["wins"] += match_result["player_4_wins"]
        player_dict[row["player_b_2"]]["losses"] += match_result["player_4_losses"]
        player_dict[row["player_b_2"]]["played"] += match_result["player_4_played"]
        player_dict[row["player_b_2"]]["rating"] += match_result["player_4_rating"]
        player_dict[row["player_b_2"]]["opponent_value_sum"] += match_result["opponent_value_B"] 
        player_dict[row["player_b_2"]]["opponent_value"] = player_dict[row["player_b_2"]]["opponent_value_sum"] / player_dict[row["player_b_2"]]["played"]
        player_dict[row["player_b_2"]]["partner_value_sum"] += match_result["partner_value_B_2"] 
        player_dict[row["player_b_2"]]["partner_value"] = player_dict[row["player_b_2"]]["partner_value_sum"] / player_dict[row["player_b_2"]]["played"]

    # Record ratings for this event
    for player_name in player_names:
        # player = create_player(name=player_name)
        player_dict[player_name]["rating_history"].append(
            {"rating": player_dict[player_name]["rating"], "event": badminton_date, "opponent_value": player_dict[player_name]["opponent_value"], "patner_value": player_dict[player_name]["partner_value"]}
        )

    # Convert dict to DataFrame
    result_df = pd.DataFrame.from_records(
        [player for player in [p for p in player_dict.values()]]
    )
    
    # Calculate opponent_value
    result_df["opponent_value"] = result_df["opponent_value"].apply(scaled_rating)

    # Calculate partner_value
    result_df["partner_value"] = result_df["partner_value"].apply(scaled_rating)

    # Sort according to rating
    result_df = result_df.sort_values(by="rating", ascending=False).reset_index(
        drop=True
    )

    # Calculate weekly rating change
    result_df["weekly_change"] = result_df["rating_history"].apply(
        get_last_week_rating_change
    )
    # Calculate event rating change
    result_df["event_change"] = result_df["rating_history"].apply(
        get_last_event_rating_change
    )
    # Calculate total rating change
    result_df["total_change"] = result_df["rating"] - 1500

    # Calculate scaled rating
    result_df["scaled_rating"] = result_df["rating"].apply(scaled_rating)

    # Set index to start at 1
    result_df.index = pd.RangeIndex(start=1, stop=len(result_df) + 1)

    # Make the decimal points more human readable
    result_df[["weekly_change", "event_change", "total_change"]] = result_df[
        ["weekly_change", "event_change", "total_change"]
    ].apply(lambda x: x.round(0).astype(int))
    result_df[["rating"]] = result_df[["rating"]].apply(lambda x: x.round(1))

    result_df = result_df[
        [
            "name",
            "played",
            "wins",
            "losses",
            "scaled_rating",
            "rating",
            # "weekly_change",
            "event_change",
            "total_change",
            "opponent_value",
            "partner_value",
        ]
    ]

    print(badminton_date)
    display(result_df)

    file_path = f"elo-scores/{badminton_date}_elo_scores.csv"
    result_df.to_csv(f"./{file_path}", index=False)
    upload_to_github(
        f"./{file_path}", commit_message=f"Upload ELO scores data for {badminton_date}"
    )


2024-10-10


,name,played,wins,losses,scaled_rating,rating,event_change,total_change,opponent_value,partner_value
1,Isha,4,3,1,5.5,1549.0,49,49,5.1,5.1
2,Kamil,4,3,1,5.5,1549.0,49,49,5.1,5.1
3,Khairul,4,3,1,5.4,1538.1,38,38,4.9,5.3
4,Suraya,4,3,1,5.4,1538.1,38,38,4.9,5.3
5,Afiqah,4,3,1,5.4,1537.3,37,37,4.9,5.2
6,Rafiq,4,3,1,5.4,1537.3,37,37,4.9,5.2
7,Shazwan,4,3,1,5.3,1529.1,29,29,5.1,5.3
8,Hazwan,4,3,1,5.3,1529.1,29,29,5.1,5.3
9,Yassier,3,2,1,5.2,1518.5,18,18,5.0,4.8
10,Alif,4,2,2,5.0,1499.3,-1,-1,4.9,4.8


File exists! Uploading...
Updated ./elo-scores/2024-10-10_elo_scores.csv in badgerminton/badgerminton-data!
2024-10-17


,name,played,wins,losses,scaled_rating,rating,event_change,total_change,opponent_value,partner_value
1,Afiqah,10,8,2,6.2,1619.0,82,119,5.0,5.2
2,Ammar,4,4,0,5.8,1582.4,82,82,5.2,5.5
3,Isha,9,6,3,5.6,1555.4,6,55,5.0,5.3
4,Kamil,4,3,1,5.5,1549.0,49,49,5.1,5.1
5,Khairul,9,6,3,5.4,1539.6,2,40,4.9,5.5
6,Suraya,4,3,1,5.4,1538.1,38,38,4.9,5.3
7,Rafiq,4,3,1,5.4,1537.3,37,37,4.9,5.2
8,Hazwan,9,6,3,5.4,1535.6,6,36,5.0,5.5
9,Rushdi,5,3,2,5.3,1533.8,34,34,5.2,4.7
10,Shazwan,9,6,3,5.3,1530.7,2,31,5.0,5.5


File exists! Uploading...
Updated ./elo-scores/2024-10-17_elo_scores.csv in badgerminton/badgerminton-data!
2024-10-31


,name,played,wins,losses,scaled_rating,rating,event_change,total_change,opponent_value,partner_value
1,Isha,16,12,4,6.4,1644.8,89,145,5.0,5.0
2,Afiqah,16,12,4,6.3,1633.1,14,133,5.0,5.3
3,Aidi,6,6,0,6.2,1621.9,122,122,4.9,4.8
4,Ammar,4,4,0,5.8,1582.4,82,82,5.2,5.5
5,Rushdi,12,7,5,5.5,1551.0,17,51,5.2,4.9
6,Kamil,4,3,1,5.5,1549.0,49,49,5.1,5.1
7,Khairul,15,9,6,5.4,1540.4,1,40,4.9,5.3
8,Rafiq,4,3,1,5.4,1537.3,37,37,4.9,5.2
9,Hazwan,9,6,3,5.4,1535.6,6,36,5.0,5.5
10,Shazwan,9,6,3,5.3,1530.7,2,31,5.0,5.5


File exists! Uploading...
Updated ./elo-scores/2024-10-31_elo_scores.csv in badgerminton/badgerminton-data!
2024-11-07


,name,played,wins,losses,scaled_rating,rating,event_change,total_change,opponent_value,partner_value
1,Afiqah,20,16,4,6.9,1693.4,60,193,5.0,5.4
2,Isha,23,17,6,6.8,1682.0,37,182,5.0,5.0
3,Khairul,21,14,7,6.1,1605.4,65,105,4.9,5.3
4,Hazwan,14,10,4,6.0,1601.7,66,102,5.1,5.4
5,Aidi,11,8,3,5.9,1588.2,-34,88,5.1,4.9
6,Ammar,10,7,3,5.8,1580.0,-2,80,5.1,5.3
7,Kamil,4,3,1,5.5,1549.0,49,49,5.1,5.1
8,Rafiq,4,3,1,5.4,1537.3,37,37,4.9,5.2
9,Rushdi,20,10,10,5.3,1530.3,-21,30,5.3,5.0
10,Suraya,12,7,5,5.2,1522.2,31,22,5.0,5.6


File exists! Uploading...
Updated ./elo-scores/2024-11-07_elo_scores.csv in badgerminton/badgerminton-data!


In [45]:
print(player_dict)

{'Isha': {'name': 'Isha', 'played': 23, 'wins': 17, 'losses': 6, 'rating': 1682.0070382711533, 'opponent_value': 1502.7980369485754, 'opponent_value_sum': 34564.35484981723, 'partner_value': 1496.35559192928, 'partner_value_sum': 34416.17861437344, 'rating_history': [{'rating': 1548.964402078087, 'event': '2024-10-10', 'opponent_value': 1508.8723371079097, 'patner_value': 1513.1716545866395}, {'rating': 1555.43981906345, 'event': '2024-10-17', 'opponent_value': 1497.4615672617342, 'patner_value': 1527.9728077362806}, {'rating': 1644.8323508393344, 'event': '2024-10-31', 'opponent_value': 1500.518791852559, 'patner_value': 1498.1127581209191}, {'rating': 1682.0070382711533, 'event': '2024-11-07', 'opponent_value': 1502.7980369485754, 'patner_value': 1496.35559192928}]}, 'Mirza': {'name': 'Mirza', 'played': 22, 'wins': 7, 'losses': 15, 'rating': 1409.0223761346836, 'opponent_value': 1516.3417987115688, 'opponent_value_sum': 33359.51957165451, 'partner_value': 1495.2495700364618, 'partner